In [1]:
import nltk, re
from nltk.stem.porter import PorterStemmer
import pandas as pd



# Reference: https://ijbnpa.biomedcentral.com/articles/10.1186/s12966-017-0525-8

# Try: need to have at least 1 key verb and 1 key noun to be classified as sedentary
# instead of relying on just 1 keyword

In [2]:
# test set
test = pd.read_json('~/Documents/PhD/HBDC/FilteredTweets/sleep_tweets_2017.12.01.20.09.json', lines=True)

test

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,None,2017-10-29 22:43:39,"[0, 12]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.245877e+17,9.245877e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",u inspire me https://t.co/VdECceNqpI,2017-10-29 22:43:39.375,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,NaN,None,2017-10-29 22:43:39,"[0, 26]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247346e+17,9.247346e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",An icon of Canadian art... https://t.co/Fxk3w5...,2017-10-29 22:43:39.431,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,NaN,None,2017-10-29 22:43:40,"[9, 32]","{u'user_mentions': [{u'id': 203123011, u'indic...",NaN,NaN,0,False,low,...,9.244376e+17,9.244376e+17,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@CTVNews https://t.co/vcQUwe5ww6,2017-10-29 22:43:40.198,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,NaN,None,2017-10-29 22:43:40,"[40, 49]","{u'user_mentions': [{u'id': 747981270, u'indic...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@rouxx21 @TalonFruitBat @Twilight_Saint &lt;33333,2017-10-29 22:43:40.311,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,NaN,None,2017-10-29 22:43:39,"[0, 62]","{u'user_mentions': [{u'id': 1565023633, u'indi...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I get A's &amp; B's and she gets D's....nuts!!...,2017-10-29 22:43:39.434,False,"{u'follow_request_sent': None, u'profile_use_b..."
5,NaN,None,2017-10-29 22:43:40,"[27, 140]","{u'user_mentions': [{u'id': 4560764232, u'indi...",NaN,"{u'display_text_range': [27, 167], u'entities'...",0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@_Nick_Nation_ @nickcarter Can believe its bee...,2017-10-29 22:43:40.770,True,"{u'follow_request_sent': None, u'profile_use_b..."
6,NaN,None,2017-10-29 22:43:40,"[0, 100]","{u'user_mentions': [{u'id': 567010052, u'indic...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Fav song 🔥 Honestly I’ve been playing this so...,2017-10-29 22:43:40.808,False,"{u'follow_request_sent': None, u'profile_use_b..."
7,NaN,None,2017-10-29 22:43:41,"[0, 0]","{u'user_mentions': [], u'symbols': [], u'hasht...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",https://t.co/dq4Ti7e1wx,2017-10-29 22:43:41.667,False,"{u'follow_request_sent': None, u'profile_use_b..."
8,NaN,"{u'type': u'Point', u'coordinates': [-66.0196,...",2017-10-29 22:43:42,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""https://untappd.com"" rel=""nofollow"">U...",It has more sweet flavour notes than most othe...,2017-10-29 22:43:42.172,False,"{u'follow_request_sent': None, u'profile_use_b..."
9,NaN,None,2017-10-29 22:43:42,"[0, 18]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247512e+17,9.247512e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Correct.That’s it. https://t.co/0jtu2oILGN,2017-10-29 22:43:42.359,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [14]:
# Verbs indicate sedentary
verbs = ['watch', 'chill', 'work', 'study', 'lying', 'recline', 'sit\s', 'read\s', 'draw', 'paint'
        , 'drive']

# Nouns indicate sedentary
nouns = ['game', 'video game', 'school', 'office', 'television', 'tv', 
         'computer', 'phone', 'iphone', 'tablet', 'ipad', 'kindle', 'book', 
         'newspaper', 'homework', 'car\s', 'plane', 'bus', 'train', 'netflix',
         'bed', 'the\ play', 'the\ show'] 



# list of sport leagues
leagues = ['nba', 'worldseri', 'nfl', 'mls', 'baseball', 'mlb', 'soccer', 'mls', 'ufc', 'nhl', 'cfl']


exclusion = ['look for work', 'hire', 'job', 'career']

In [15]:
# French version
from googletrans import Translator
translator = Translator()

verbs_fr = [translator.translate(x, dest='fr').text for x in verbs]
verbs_fr

nouns_fr = [translator.translate(x, dest='fr').text for x in nouns]
nouns_fr  

[u'Jeu',
 u'jeu vid\xe9o',
 u'\xe9cole',
 u'Bureau',
 u't\xe9l\xe9vision',
 u'la t\xe9l\xe9',
 u'ordinateur',
 u't\xe9l\xe9phone',
 u'iphone',
 u'tablette',
 u'ipad',
 u'kindle',
 u'livre',
 u'journal',
 u'devoirs',
 u'car\\s',
 u'avion',
 u'bus',
 u'train',
 u'netflix',
 u'lit',
 u'le jeu',
 u'le spectacle']

In [16]:
# stem forms

porter = nltk.PorterStemmer()

verbs_stemmed = [porter.stem(item) for item in verbs]
#verbs_stemmed

# stemming nouns doesnt make sense
#nouns_stemmed = [porter.stem(item) for item in nouns]
#verbs_stemmed


In [17]:
# Regex pattern
verbs_regex = "\s(" + '|'.join(verbs + verbs_stemmed + verbs_fr) + ')'
verbs_regex = re.sub(' ', '\ ', verbs_regex)


nouns_regex = "\s(" + '|'.join(nouns + nouns_fr + leagues) + ')'
#nouns_regex = '\s' + '|'.join(nouns + nouns_stemmed)
#nouns_regex = re.sub(' ', '\ ', nouns_regex)


exclusion_regex = "\s(" + '|'.join(exclusion) + ')'


In [18]:
# tokenize each tweets
# look for match in keyword
# but have to include word boundary

for i, item in enumerate(test.text):
    #print item 
    lowered = item.lower()
    
    tokens = nltk.word_tokenize(lowered)

    stemmed_tokens = [porter.stem(x) for x in tokens]

    #print stemmed_tokens

    verbs_count = len(re.findall(verbs_regex, ' '.join(stemmed_tokens)))
    #print re.findall(verbs_regex, ' '.join(stemmed_tokens))
    #print verbs_count

    noun_count = len(re.findall(nouns_regex, ' '.join(stemmed_tokens)))
    #print re.findall(nouns_regex, ' '.join(stemmed_tokens))
    #print noun_count
    #print '\n'
    
    exlusion_count = len(re.findall(exclusion_regex, ' '.join(stemmed_tokens)))

    if verbs_count >=1 and noun_count >=1 and exlusion_count == 0:
    	test.at[i, 'sedentary'] = 1
    	print item
    	#print stemmed_tokens
    	print re.findall(verbs_regex, ' '.join(stemmed_tokens))
    	print re.findall(nouns_regex, ' '.join(stemmed_tokens))
    	print '\n'
    else:
    	test.at[i, 'sedentary'] = 0


Even though I slept from 8pm to 530 am and had a nap after work, I'm still exhausted and I'm going back to bed now… https://t.co/PpL1bztSny
[u'work']
[u'bed']


Okie dokie... Now that I’m just lying in bed; refusing to move because I’m warm as fuck and wrapped like a burrito.… https://t.co/I0RGGJXa7s
[u'lie']
[u'bed']


Everytime I drive my little sis somewhere she always wants to listen to Migos
[u'drive']
[u'lit']


J’suis en train de manquer sûrement le plus nice épisode de Od bali pcq je travaille 😩
[u'travail']
[u'train']


Yes., these are professional race car drivers @MartinsvilleSwy 😂🙄
[u'drive']
[u'car ']


I hate it when am watching a trailer on TV 📺 for what looks like a cool movie 🎥 but turns out to be a new fucking video game 🎮.
[u'watch']
[u'tv', u'video game']


Drinking beer 🍺 watching #NFLSunday = my happy place!
My beloved #Seahawks &amp; #FF are pure entertainment today 🙌🏼🏈
[u'watch']
[u'nfl']


Why yes drivers of Halifax, my car broke down just before the bridge. La

[u'watch']
[u'bed']


@Lord_Oni is live on #twitch! Come chill!
https://t.co/CkXgixyHZg
@SupStreamers @WE_RTs @GamerRetweeters @TwitchSharer @GamerRetweeters
[u'chill']
[u'game', u'game']


Some nice bottom 6 work tonight. Kudos to Lemieux, gets his 1st #NHLJets goal.
[u'work']
[u'nhl']


We have new Store Hours! No, Joe won’t be working them all lol. Please keep booking online at… https://t.co/ensEG9Xxdj
[u'work']
[u'book']


When you finally get one of your friends to read one of your fave books and you can’t wait for them to finish. #hurryup @Selecta_Kenrock 🤓🤓
[u'read ']
[u'book']


Le'Veon Bell is possibly the most patient running back, he'll literally sit there until he sees an opening
[u'sit ']
[u'lit']


@calummadell @DanBarberTF That will be the Sunday I boycott the NFL and take up watching Ultimate Knit Off on my weekends.
[u'watch']
[u'nfl']


Did I do any homework this weekend? No. Did I binge watch Stranger Things? Yes. Do I care? Definitely not.
[u'watch']
[u'homework']



@cdnbeer I haven’t seen a TV commercial in years (refuse to watch anything live), but I can well imagine they’d be… https://t.co/HISaP5J1F6
[u'watch']
[u'tv']


@SuHolmesMcKagan Watching the show on Toronto, Ontario 🇨🇦 vidéo @JordanFPerform Thank You! @gunsnroses #GNRToronto… https://t.co/QAIoszsr8L
[u'watch']
[u'the show']


I work till' 9 on Halloween meaning I can go out and get lit but I don't have a costume lmao
[u'work']
[u'lit']


@CatherineZed only have 2 songs left on my play list I picked for the night.  So you know what I will be watching #Astros #WorldSeries
[u'watch']
[u'worldseri']


my social studies friend will buy me real hamster if i’ll go with them at summer school yo 😂
[u'studi']
[u'school']


Time to watch some #WorldSeries ball #Astros #WorldSeries
[u'watch']
[u'worldseri', u'worldseri']


Bien heureux de ne pas travailler demain matin je peux continuer à regarder ce match de fou! ⚾ #WorldSeries
[u'travail']
[u'worldseri']


@20committee @RiskyLiberal Only 51%??? 

[u'watch']
[u'worldseri', u'game']


Anyone else waiting to watch @TheWalkingDead on pvr? Instant classic #WorldSeries2017 @mlb @Dodgers @astros @AMC_TV
[u'watch']
[u'worldseri', u'mlb']


Thankful for @Shaw #FreeRangeTV so I can watch this never ending game even when laid up.
[u'watch']
[u'game']


@JeffTweedy It’s 1:05am...have to work tomorrow morning (today) and I’m busting a gut reading this!
[u'work', u'read ']
[u'bus']


Now even i am watching this baseball game
[u'watch']
[u'game']


When your DVR kicks in and boots you out of watching the 9th inning... #worldseries
[u'watch']
[u'worldseri']


Craziest baseball game I've ever watch.  Shits WILD
[u'watch']
[u'game']


I sure am glad I'm watching this game on the west coast.
[u'watch']
[u'game']


I’ve got a busy day tomorrow, but I can’t stop watching this ballgame. Tied 12:12 in the 9th. #MLB
[u'watch']
[u'bus', u'mlb']


And productivity at workplaces and classrooms plummet tomorrow morning with that catch #extras #ThisTeam #W

When you’re sitting in prison you actually want to go to court. But when you’re home watching tv you don’t.
[u'sit ', u'watch']
[u'tv']


"Freedom in Christ (Sabbath School Study Hour"
Pastor Doug Batchelor

https://t.co/CKkOrl74m9
Share, WORK FOR JESUS, FRIEND
[u'studi', u'work']
[u'school']


La Formule E a nui à trois commerçants sur quatre selon une étude obtenue par @tvanouvelles… https://t.co/xzXeMmcFEi
[u'\xe9tude']
[u'tv']


"Living by the Spirit (Sabbath School Study Hour"
Pastor Doug Batchelor

https://t.co/ZeWXFCc8Nz
Share, WORK FOR JESUS, FRIEND
[u'studi', u'work']
[u'school']


My plan after this early Monday morning shift is to get right back into bed until I have to work later tonight as well 😴
[u'work']
[u'bed']


Not gonna lie, it’s raining a little here in Ottawa and a bit cool. #NoSnowThough https://t.co/nuRyHu5TSH
[u'lie']
[u'lit']


"The Two Covenants (Sabbath School Study Hour"
Pastor Doug Batchelor

https://t.co/yShToM9aqt
Share, WORK FOR JESUS, FRIEND
[u'studi',

...

Do I do my makeup just to watch spooky movies alone in bed? Yes or yes????
[u'watch']
[u'bed']


Yikes!! Probably a good thing this driver got out of his car to flag people down. https://t.co/ON2zcumiBj
[u'drive']
[u'car ']


Want to know how to make English Department Heads smile after a long day? Give them new books to read! #hcdsbcurr… https://t.co/MbSNViPT5d
[u'read ']
[u'book']


I want to open a business called #MysteriousWays just so I can put a sign in the window that says "Your Lord doesn't work here!"
[u'work']
[u'bus']


Want to know how to make English Department Heads smile after a long day? Give them new books to read! #hcdsbcurr… https://t.co/Eff3YrUjZJ
[u'read ']
[u'book']


So I messaged up a friend back from High School on Instagram, he freakin saw my message &amp; left me on read.... All I… https://t.co/geWnOTaz17
[u'read ']
[u'school']


@dashhaib when u a good ass workout is being thrown around in the graves crowd and laughing at literally everything… https://t.co/



It was so awesome watching everyone planting, we are on our way to create little stewards of our earth… https://t.co/fB6zgCDTXS
[u'watch']
[u'lit']


I DUG MY KEY INTO THE SIDE 
OF HIS PRETTY LITTLE SOUPED-UP-FOUR-WHEEL DRIVE
CARVED MY NAME INTO HIS LEATHER SEATS
I… https://t.co/oHU5z1QdvI
[u'drive']
[u'lit']


On some awesome @devincicycles  #hatchets testing out the @tacx &amp; @CycleOps  direct drive smart trainers!

#gozwift… https://t.co/PaadRuqnEk
[u'drive']
[u'train']


Baptist, or baptized? I have read the bible, i grew up in a catholic school and going to church. so, call me a dumb… https://t.co/UKp3E1bLDe
[u'read ']
[u'school']


Currently obsessed with @Netflix_CA Confession Tapes! I’ve been binge watching this series all day! #Netflix… https://t.co/yglp6QuHzs
[u'watch']
[u'netflix', u'netflix']


👏 👏 👏 Special congrats to [busy Playwright] @theatreheather who is workshopping her play Silver Fish at #edna's thi… https://t.co/0CqD4v2aFM
[u'work']
[u'bus']


@nutrition_facts

@SarahOttawa Is your daughter going into the trades, Sarah? Maybe our girls could start a business doing work for d… https://t.co/GZK5aaV3Q2
[u'work']
[u'bus']


@axematty @EdBrown19 The last book I "read" was by Lena Dunham and I still have like 80 pages to go. It's been years.
[u'read ']
[u'book']


I suppose it’s a good thing I ain’t watching the game lol
[u'watch']
[u'game']


Phenomenal to see Mayors @JohnTory, @LindaJeffrey and @BonnieCrombie work together to attract businesses to invest… https://t.co/SHqiiu7V40
[u'work']
[u'bus']


Vegan ALERT 4 all Vegans as U learn to MASTER Vegan "Lifestyle" Google predicted "Vegan Revolution" 1st read book b… https://t.co/Gyxoi7PnBf
[u'read ']
[u'book']


@tom1robson We will hear more about this painting, Very little of it is real.
[u'paint']
[u'lit']


@DanielPink's "Drive" is absolutely phenomenal. This book should be mandatory reading for any #management. #motivation #TypeI #Drive
[u'drive', u'read ', u'drive']
[u'book']


Superman drawin

@Kaepernick7 time to come to the CFL. You’d be a great @Wpg_BlueBombers , but honestly you’d be great to watch in any uni.
[u'watch']
[u'cfl']


I haven't watched a world cup game yet bc of school 🙃
[u'watch']
[u'game', u'school']


Go back in game - nothing. Alt tabbed out, read maps don't work anymore (?) and heard, clear as day, "dude, there's… https://t.co/qDh9ZmTDZh
[u'read ', u'work']
[u'game']


Slay every night and day, read idiots to filth, leave dead weight behind and wear a crown to bed 😁 https://t.co/f4WxQN0fka
[u'read ']
[u'bed']


Avon Rep Tip: And she stole the show… Top off your holiday outfit with this luxe Chill Factor Stole from mark. https://t.co/vPk6wp0kzX
[u'chill']
[u'the show']


Yes I am home from work. Yes I saw a pic of @AluraJenson she tweeted from a shoot that almost lit my laptop on fire… https://t.co/Mtc7RYCNCM
[u'work']
[u'lit']


@glaxdal1 @ahs_scn @timbaron70 @slitvinch1 Agree, I trained there for 2 years, great people doing great work! Foothills rocks

That's love: @selenagomez and her dog watching @justinbieber play a little pick-up hockey last night in L.A. https://t.co/fh5RaTEtc5
[u'watch']
[u'lit']


Changing it up a little.

#pullups #exercise #workouts #workout #fitness #fitnessjunkie… https://t.co/r2USAR4xSf
[u'work', u'work']
[u'lit']


@DSIG_Samantha Morning Happy Thurs their last game on road then they are home on the weekend very exciting u watchi… https://t.co/8nwhkGEHvA
[u'watch']
[u'game']


Yay! Can't wait to watch this event at the pyeongchang winter games #HK95240 https://t.co/7YpFqUloA1
[u'watch']
[u'game']


@DSIG_Katie Morning Happy Thurs u watching the game tonight the last away one and their back on the weekend excitin… https://t.co/NeMIsWiLpV
[u'watch']
[u'game']


Can my brand new phone work properly or is that just too convenient, y’all my texts don’t work it freezes thanks iPhone X
[u'work', u'work']
[u'phone']


@DSIG_Rheagan Morning Happy Thurs the last away game tonight u watching it then there back home 

In [19]:
test[test.sedentary == 1]

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user,sedentary
133,NaN,None,2017-10-29 22:45:35,"[0, 140]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,"{u'display_text_range': [0, 123], u'entities':...",0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",Even though I slept from 8pm to 530 am and had...,2017-10-29 22:45:35.606,True,"{u'follow_request_sent': None, u'profile_use_b...",1.0
156,NaN,None,2017-10-29 22:46:01,"[0, 140]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,"{u'display_text_range': [0, 138], u'entities':...",0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Okie dokie... Now that I’m just lying in bed; ...,2017-10-29 22:46:01.853,True,"{u'follow_request_sent': None, u'profile_use_b...",1.0
270,NaN,None,2017-10-29 22:47:52,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Everytime I drive my little sis somewhere she ...,2017-10-29 22:47:52.519,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
457,NaN,None,2017-10-29 22:51:20,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",J’suis en train de manquer sûrement le plus ni...,2017-10-29 22:51:20.998,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
535,NaN,None,2017-10-29 22:52:20,NaN,"{u'user_mentions': [{u'id': 21672310, u'indice...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...","Yes., these are professional race car drivers ...",2017-10-29 22:52:20.093,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
647,NaN,None,2017-10-29 22:54:18,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I hate it when am watching a trailer on TV 📺 ...,2017-10-29 22:54:18.797,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
963,NaN,None,2017-10-29 22:59:15,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Drinking beer 🍺 watching #NFLSunday = my happ...,2017-10-29 22:59:15.245,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
1062,NaN,None,2017-10-29 23:00:28,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...","Why yes drivers of Halifax, my car broke down ...",2017-10-29 23:00:28.800,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
1172,NaN,None,2017-10-29 23:02:07,NaN,"{u'user_mentions': [{u'id': 18336787, u'indice...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",If you aren’t watching @HoustonTexans vs @Seah...,2017-10-29 23:02:07.049,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
1189,NaN,None,2017-10-29 23:02:16,"[19, 107]","{u'user_mentions': [{u'id': 75332107, u'indice...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","@nikgraf @rustlang Coincidently, I TIL'd this ...",2017-10-29 23:02:16.884,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
